<a href="https://colab.research.google.com/github/vijaykumarharakuni/Deeplearning/blob/main/ImageClassification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data (1)/test_set.zip','r')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
folder='/content/training_set/cats/'
for i in range(9):
  plt.subplot(330+1+i)
  filename=folder+'cat.'+str(i+1)+'.jpg'
  image=imread(filename)
  plt.imshow(image)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
folder='/content/training_set/dogs/'
for i in range(9):
  plt.subplot(330+1+i)
  filename=folder+'cat.'+str(i+1)+'.jpg'
  image=imread(filename)
  plt.imshow(image)
plt.show()

In [ ]:
# generate more images by applying some tranformations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 horizontal_flip=True,
                                 vertical_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

Crate Train Set & test set

In [ ]:
training_set=train_datagen.flow_from_directory('/content/training_set',
                                               target_size=(150,150),
                                               class_mode='binary',
                                               batch_size=32)


In [ ]:
test_set=test_datagen.flow_from_directory('/content/test_set',
                                               target_size=(150,150),
                                               class_mode='binary',
                                               batch_size=32)

In [ ]:
#Print class labels

print(training_set.class_indices)


Create CNN

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
print('Imported all libraries')

Imported all libraries


In [3]:
# create architecture
model=Sequential()
# add first CNN layer
model.add(Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
# add second CNN layer
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
# third cnn layer
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
# flatten layer
model.add(Flatten())
# FCN
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
print('model created')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


model created


In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 17, 17, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 73984)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      37,880,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,251,649 (145.92 MB)

 Trainable params: 38,251,649 (145.92 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print('compiled')

compiled


(H-K+2P)/S + 1 - for 148

In [6]:
history=model.fit(training_set,
                  epochs=10,
                  validation_data=test_set)

NameError: name 'training_set' is not defined

In [ ]:
# evaluate the model
score=model.evaluate(test_set)
print('Test Loss:',score[0])
print('Test Accuracy:',score[1])


In [ ]:
# evaluate for training
score=model.evaluate(training_set)
print('Train Loss:',score[0])
print('Train Accuracy:',score[1])

In [ ]:
# plot the accuracy
plt.plot(history.history['accuracy'],label='train accuracy')
plt.plot(history.history['val_accuracy'],label='test accuracy')
plt.legend()
plt.show()

In [ ]:
# plot the losses
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='test loss')
plt.legend()
plt.show()

In [ ]:
# pred on new data
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import numpy as np
test_image=load_img('/content/test_set/cats/cat.4001.jpg')
plt.imshow(test_image)
plt.axis('off')
plt.show()
test_image=load_img('/content/test_set/cats/cat.4001.jpg',target_size=(64,64))
test_image=img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
print(result)
if result[0][0]==1:
  print('its a cat')
else:
  print('its a dog')

Challenges
1.Lack of Massive dataset
2.Computation Power
3.